## 1. Install Dependencies and Import Libraries

Install LlamaIndex and other dependencies.

In [ ]:
!pip install -q gcsfs llama-index tqdm

Import libraries.

In [ ]:
import os

import openai
import pandas as pd
import phoenix as px
from llama_index.llms import OpenAI
from llama_index.tools import FunctionTool
from phoenix.experimental.callbacks.llama_index_trace_callback_handler import (
    OpenInferenceTraceCallbackHandler,
)
from tqdm import tqdm

# TODO: this entirely breaks phoenix
# nest_asyncio.apply()


pd.set_option("display.max_colwidth", 1000)

In [ ]:
# launch phoenix
px.launch_app()

In [ ]:
# Define very simple calculator tools for our agent


def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## 2. Configure Your OpenAI API Key

In [ ]:
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## Initialize the OpenAI Agent

In [ ]:
# Initialize the agent
from llama_index.agent import OpenAIAgent
from llama_index.callbacks import CallbackManager
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-0613")
cb_handler = OpenInferenceTraceCallbackHandler()
callback_manager = CallbackManager(handlers=[cb_handler])
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True, callback_manager=callback_manager
)

In [ ]:
response = agent.chat("What is (121 * 3) + 42?")

In [ ]:
queries = [
    "What is (121 * 3) + 42?",
    "what is 3 * 3?",
    "what is 4 * 4?",
    "what is 75 * (3 + 4)?",
]
for query in tqdm(queries):
    response = agent.chat(query)

In [ ]:
from phoenix import TraceDataset
from phoenix.trace.span_json_encoder import spans_to_jsonl
from phoenix.trace.trace_dataset import TraceDataset

ds = TraceDataset.from_spans(list(cb_handler.get_spans()))

ds.dataframe.head()

In [ ]:
# Dump the contents to a file for safe keeping
from phoenix.trace.span_json_encoder import spans_to_jsonl

export_trace = False
if export_trace:
    with open("trace.jsonl", "w") as f:
        f.write(spans_to_jsonl(callback_handler._tracer.span_buffer))